# Day 23

In [182]:
with open('23_data') as f:
    lines = [line.strip() for line in f.readlines()]

In [183]:
openCells = ['.','>','<','v','^']
def countNeighbors(i,j,lines):
    count = 0
    if i-1 >= 0:
        count += lines[i-1][j] in openCells
    if j-1 >= 0:
        count += lines[i][j-1] in openCells
    if i+1 < len(lines):
        count += lines[i+1][j] in openCells
    if j+1 < len(lines[0]):
        count += lines[i][j+1] in openCells
    return count

In [184]:
import sys, itertools

class Node:
    nextId = itertools.count()

    def __init__(self, i, j):
        self.i = i
        self.j = j
        self.id = next(Node.nextId)
        self.adjacencyList = []
    
    def __hash__(self):
        return hash((self.i, self.j))
    
    def __eq__(self, other):
        print(self)
        print(other)
        return self.i == other.i and self.j == other.j
    
    def __repr__(self):
        return f'N{self.id}:{self.i}.{self.j}'

def buildGraph(lines):
    nodes = {(0,1):Node(0,1)}
    for i in range(len(lines)):
        for j in range(len(lines[0])):
            if lines[i][j] == '#':
                continue
            n = countNeighbors(i,j,lines)
            if n > 2:
                nodes[(i,j)] = Node(i,j)
    nodes[(len(lines)-1,len(lines[0])-2)] = Node(len(lines)-1,len(lines[0])-2)
    edges = []
    for (i,j) in nodes:
        for d in dirMap:
            di,dj = dirMap[d]
            #  print("Starting walk:", i, j, d)
            (dist, destNode) = walkPath(i+di, j+dj, invMap[d],lines,nodes)
            if destNode:
                # print("Found node: ",destNode, dist)
                edges += [(nodes[(i,j)], destNode, dist+1)]
    
    # P1
    # return nodes, edges
    
    # P2
    adjGraph = [[0]*len(nodes) for _ in range(len(nodes))]
    for (n1, n2, dist) in edges:
        adjGraph[n1.id][n2.id] = dist
        adjGraph[n2.id][n1.id] = dist
    return adjGraph
            


dirMap = {'s':(1,0),'n':(-1,0),'w':(0,-1),'e':(0,1)}
invMap = {'s':'n','n':'s','e':'w','w':'e'}
dirChars = {'s':'v', 'n':'^','e':'>', 'w': '<'}
def walkPath(i, j, fromDir, lines, nodes):
    if (i,j) in nodes:
        return (0, nodes[(i,j)])
    if i<0 or j<0 or i>=len(lines) or j >= len(lines[0]):
        return (1000000, None)
    if lines[i][j] == '#':
        return (1000000, None)
    if lines[i][j] == dirChars[fromDir]:
        return (1000000, None)
    nextSteps = []
    for d in dirMap:
        if d == fromDir:
            continue
        (di, dj) = dirMap[d]
        ni, nj = i+di,j+dj
        if ni<0 or ni > len(lines) or nj<0 or nj>len(lines):
            continue
        if lines[ni][nj] not in ['.', '>', '<', 'v', '^']:
            continue
        nextSteps += [(ni, nj, d)]
    
    if len(nextSteps) == 0:
        return (100000000, None)
    elif len(nextSteps) == 1:
        (ni, nj, direction) = nextSteps[0]
        (dist, destNode) = walkPath(ni, nj, invMap[direction], lines, nodes)
        return (1+dist, destNode)

In [137]:
nodes,edges = buildGraph(lines)
print(nodes)
print(edges)

{(0, 1): N0:0.1, (5, 17): N1:5.17, (5, 67): N2:5.67, (9, 81): N3:9.81, (13, 33): N4:13.33, (19, 111): N5:19.111, (29, 19): N6:29.19, (29, 137): N7:29.137, (33, 41): N8:33.41, (33, 109): N9:33.109, (35, 59): N10:35.59, (39, 87): N11:39.87, (53, 99): N12:53.99, (59, 11): N13:59.11, (61, 137): N14:61.137, (63, 41): N15:63.41, (63, 55): N16:63.55, (65, 81): N17:65.81, (77, 31): N18:77.31, (79, 101): N19:79.101, (81, 13): N20:81.13, (81, 81): N21:81.81, (83, 135): N22:83.135, (87, 55): N23:87.55, (101, 29): N24:101.29, (101, 63): N25:101.63, (103, 87): N26:103.87, (111, 101): N27:111.101, (111, 131): N28:111.131, (113, 11): N29:113.11, (125, 63): N30:125.63, (129, 103): N31:129.103, (129, 127): N32:129.127, (131, 87): N33:131.87, (133, 43): N34:133.43, (140, 139): N35:140.139}
[(N0:0.1, N1:5.17, 117), (N1:5.17, N6:29.19, 142), (N1:5.17, N4:13.33, 104), (N2:5.67, N10:35.59, 234), (N2:5.67, N3:9.81, 98), (N3:9.81, N11:39.87, 148), (N3:9.81, N5:19.111, 280), (N4:13.33, N8:33.41, 124), (N4:13.3

In [142]:
def longestPath(nodes, edges):
    # Assuming DAG
    predecessors = {n:[] for n in list(nodes.values())}
    successors = {n:[] for n in list(nodes.values())}
    outboundIndices = {n:[] for n in list(nodes.values())}
    inboundIndices = {n:[] for n in list(nodes.values())}

    for i in range(len(edges)):
        (n1, n2, dist) = edges[i]
        # print(edges[i])
        # print(n2)
        predecessors[n2] += [n1]
        successors[n1] += [n2]
        outboundIndices[n1] += [i]
        inboundIndices[n2] += [i]
        
    processed = set()
    toProcess = [nodes[(0,1)]]
    edgeEs = [0]*len(edges)
    
#     print(edges)
#     print(inboundIndices)
    
    while len(toProcess) > 0:
        nextNode = toProcess.pop(0)
        processed.add(nextNode)
        maxE = max([edgeEs[i] for i in inboundIndices[nextNode]] + [0])
        # print(maxE)
        for i in range(len(successors[nextNode])):
            edgeEs[outboundIndices[nextNode][i]] = maxE + edges[outboundIndices[nextNode][i]][2]
            if (len(set(predecessors[successors[nextNode][i]]).intersection(processed))
               == len(set(predecessors[successors[nextNode][i]]))):
                toProcess += [successors[nextNode][i]]
            
    print(max(edgeEs))

In [143]:
longestPath(nodes, edges)

2210


### Part 2

In [185]:
adjGraph = buildGraph(lines)

In [186]:
adjGraph

[[0,
  117,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [117,
  0,
  0,
  0,
  104,
  0,
  142,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  98,
  190,
  0,
  0,
  0,
  0,
  0,
  234,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  98,
  0,
  0,
  280,
  0,
  0,
  0,
  0,
  0,
  148,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  104,
  190,
  0,
  0,
  0,
  0,
  0,
  124,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  280,
  0,
  0,
  0,
  376,
  0,
  28,
  0,
  0,
  0,
 

In [187]:
from copy import deepcopy

In [195]:
maxPath = 0

def dfs_recurse(adjGraph, curNode, visited, pathLength):
    global maxPath
    # print(curNode, visited, pathLength)
    if curNode == len(adjGraph) - 1:
        # print("Breaking")
        # print(pathLength)
        if pathLength > maxPath:
            print(pathLength)
            maxPath = max(maxPath, pathLength)
        return pathLength
        
    children = [i for i in range(len(adjGraph)) if adjGraph[curNode][i] != 0]
    children = list(filter(lambda x: x not in visited, children))
    # print(children)
    if len(children) == 0:
        return 0
    vNew = deepcopy(visited)
    vNew.add(curNode)
    pathLens = [dfs_recurse(adjGraph, child, vNew, pathLength+adjGraph[curNode][child]) for child in children]
    return max(pathLens)

In [196]:
print(dfs_recurse(adjGraph, 0, set(), 0))
print(maxPath)

5006
5310
5346
5630
5914
5950
6058
6166
6274
6362
6414
6490
6522


KeyboardInterrupt: 

In [192]:
len(adjGraph)

36

# Day 24

In [23]:
with open('24_data') as f:
    lines = f.readlines()

In [24]:
points = []
for line in lines:
    point, velocity = line.strip().split(' @ ')
    (px, py, pz) = point.split(', ')
    (px, py, pz) = int(px), int(py), int(pz)
    vx, vy, vz = velocity.split(', ')
    vx, vy, vz = int(vx), int(vy), int(vz)
    points += [(px, py, pz, vx, vy, vz)]

In [25]:
points

[(327367788702047, 294752664325632, 162080199489440, 20, 51, 36),
 (349323332347395, 429135322811787, 397812423558610, -96, -480, -782),
 (342928768632768, 275572250031810, 310926883862869, -69, 104, -510),
 (308103110384633, 244954649487980, 207561383118617, 220, 463, -401),
 (340999127475480, 254564767726572, 208239482774805, -9, 429, -458),
 (343240821178976, 142303638369464, 376763854620819, -104, 127, -12),
 (326497806237555, 369602061347187, 175400063083000, 116, -484, -232),
 (224914172513640, 232860203514162, 395775420910315, 19, 21, -15),
 (397260347804943, 316187216997846, 479392317394901, -210, -63, -525),
 (284365134183595, 277589471545153, 192832852241563, 158, 92, -31),
 (278215996681754, 276175917103936, 296821842607218, 41, 19, -98),
 (190366346246272, 260134954974952, 337585285708597, 107, 11, -25),
 (312672539704038, 305065464648261, 148709847682799, 57, -12, 133),
 (345595901783463, 296598876846453, 202976287684123, -78, 24, -95),
 (206227836852120, 142096590171452, 

In [26]:
lines = []
for (px, py, pz, vx, vy, vz) in points:
    slope = vy / vx
    b = py - slope * px
    lines += [(slope, b)]

In [27]:
lines

[(2.55, -540035196864587.75),
 (5.0, -1317481338925188.0),
 (-1.5072463768115942, 792450394058011.0),
 (2.1045454545454545, -403462351003315.75),
 (-47.666666666666664, 1.6508856510724452e+16),
 (-1.2211538461538463, 561453487309175.1),
 (-4.172413793103448, 1731886011510778.5),
 (1.105263157894737, -15729145053545.406),
 (0.3, 197009112656363.12),
 (0.5822784810126582, 112009773159768.56),
 (0.4634146341463415, 147246552788001.22),
 (0.102803738317757, 240564582930942.72),
 (-0.21052631578947367, 370891262480690.06),
 (-0.3076923076923077, 402936077395210.9),
 (2.3015873015873014, -332554780361205.1),
 (-2.261904761904762, 898150704470316.9),
 (-2.707317073170732, 1164255205628577.5),
 (-1.633587786259542, 864422101275628.6),
 (-2.8333333333333335, 1293901461471399.0),
 (-0.8333333333333334, 597983683857922.5),
 (3.5238095238095237, -613800146373458.5),
 (1.736842105263158, -258062693346038.56),
 (-2.0, 797318357284380.0),
 (-1.1076923076923078, 601585936166708.5),
 (2.297297297297297

In [28]:
intersects = []
for i in range(len(lines)-1):
    for j in range(i+1, len(lines)):
        (m1, b1) = lines[i]
        (m2, b2) = lines[j]
        if m1 == m2 and b1 != b2:
            intersects += [None]
        elif m1 == m2:
            print("Same line!")
            intersects += [(0,0)]
        else:
            x_intersect = (b2 - b1) / (m1 - m2)
            y_intersect = m1 * x_intersect + b1
            intersects += [(x_intersect, y_intersect, i, j)]

In [29]:
intersects

[(317324955943102.1, 269143440790322.62, 0, 1),
 (328421167257222.0, 297438779641328.25, 0, 2),
 (306592102953876.0, 241774665667796.0, 0, 3),
 (339506638717339.06, 325706731864626.75, 0, 4),
 (292082669949187.5, 204775611505840.38, 0, 5),
 (337962118711903.6, 321768205850766.4, 0, 6),
 (362907649705275.3, 385379309883864.25, 0, 7),
 (327575248675978.2, 295281687259156.6, 0, 8),
 (331370554081210.4, 304959716042498.6, 0, 9),
 (329381083994811.8, 299886567322182.4, 0, 10),
 (318977187084673.56, 273356630201329.75, 0, 11),
 (329982892803818.5, 301421179785149.4, 0, 12),
 (329976488973819.2, 301404850018651.1, 0, 13),
 (835224679853872.5, 1589787736762787.0, 0, 14),
 (298880790975091.5, 222110820121895.5, 0, 15),
 (324174931580699.44, 286610878666195.75, 0, 16),
 (335706424699148.56, 316016186118241.0, 0, 17),
 (340669348297706.56, 328671641294563.9, 0, 18),
 (336360260312072.0, 317683466931195.75, 0, 19),
 (75748847906419.83, -346875634703217.2, 0, 20),
 (346762302061646.4, 3442086733926

In [30]:
minVal = 200000000000000
maxVal = 400000000000000
count = 0
for k in range(len(intersects)):
    if not intersects[k]:
        # print(i, j, "None")
        continue
    (x, y, i, j) = intersects[k]
    if x < minVal or y < minVal or x > maxVal or y > maxVal:
        # print(i,j,"Out of bounds")
        continue
    if (((x-points[i][0]) / points[i][3]) < 0 or
        ((x-points[j][0]) / points[j][3]) < 0 or
        ((y-points[i][1]) / points[i][4]) < 0 or
        ((y-points[j][1]) / points[j][4]) < 0):
        # print(i,j,"In past")
        continue
    count += 1
print(count)

28266


### Part 2

In [61]:
from sympy import Symbol, Equality
from sympy.solvers import solve

In [62]:
x,y,z,a,b,c = Symbol('x'),Symbol('y'),Symbol('z'),Symbol('a'),Symbol('b'),Symbol('d'),

In [63]:
eqs = [
    (x-327367788702047)/(20-a) - (y-294752664325632)/(51-b),
    (x-327367788702047)/(20-a) - (z-162080199489440 )/(36-c),
    (y-294752664325632)/(51-b)-(z-162080199489440 )/(36-c),
    (x-349323332347395)/(-96-a) - (y-429135322811787)/(-480-b),
    (x-349323332347395)/(-96-a) - (z-397812423558610 )/(-782-c),
    (y-429135322811787)/(-480-b) - (z-397812423558610 )/(-782-c),
    (x-342928768632768)/(-69-a)- (y-275572250031810)/(104-b),
    (x-342928768632768)/(-69-a) - (z-310926883862869)/(-510-c),
    (y-275572250031810)/(104-b) - (z-310926883862869)/(-510-c)
]

In [65]:
soln = solve(eqs, [x,y,z,a,b,c])[0]

In [66]:
soln

(354954946036320, 318916597757112, 112745502066835, -117, -69, 281)

In [67]:
sum(soln[:3])

786617045860267

# Day 25

In [54]:
with open('25_data') as f:
    lines = [line.strip() for line in f.readlines()]

In [55]:
def buildGraph(lines):
    nodeNames = set()
    for line in lines:
        (f, rest) = line.split(': ')
        nodeNames.add(f)
        rest = rest.split()
        nodeNames = nodeNames.union(set(rest))
    nodeList = list(nodeNames)
    nodeIds = {nodeList[i]: i for i in range(len(nodeList))}
    graph = [[0]*len(nodeIds) for _ in range(len(nodeIds))]
    for line in lines:
        (f, rest) = line.split(': ')
        i = nodeIds[f]
        for r in rest.split():
            j = nodeIds[r]
            graph[i][j] = 1
            graph[j][i] = 1
    for i in range(len(graph)):
        print(nodeList[i], graph[i])
    
    return graph


In [56]:
grid = buildGraph(lines)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [57]:
import collections


class Graph:
    """
    This class represents a directed graph using
    adjacency matrix representation.
    """

    def __init__(self, graph):
        self.graph = graph  # residual graph
        self.row = len(graph)

    def bfs(self, s, t, parent):
        """
        Returns true if there is a path from
        source 's' to sink 't' in residual graph.
        Also fills parent[] to store the path.
        """

        # Mark all the vertices as not visited
        visited = [False] * self.row

        # Create a queue for BFS
        queue = collections.deque()

        # Mark the source node as visited and enqueue it
        queue.append(s)
        visited[s] = True

        # Standard BFS loop
        while queue:
            u = queue.popleft()

            # Get all adjacent vertices of the dequeued vertex u
            # If an adjacent has not been visited, then mark it
            # visited and enqueue it
            for ind, val in enumerate(self.graph[u]):
                if (visited[ind] == False) and (val > 0):
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u

        # If we reached sink in BFS starting from source, then return
        # true, else false
        if not visited[t]:
            print('Broken component size:', sum(visited), len(visited)-sum(visited))
        return visited[t]

    # Returns the maximum flow from s to t in the given graph
    def edmonds_karp(self, source, sink):
        # This array is filled by BFS and to store path
        parent = [-1] * self.row

        max_flow = 0  # There is no flow initially

        # Augment the flow while there is path from source to sink
        while self.bfs(source, sink, parent):
            # Find minimum residual capacity of the edges along the
            # path filled by BFS. Or we can say find the maximum flow
            # through the path found.
            path_flow = float("Inf")
            s = sink
            while s != source:
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            # Add path flow to overall flow
            max_flow += path_flow

            # update residual capacities of the edges and reverse edges
            # along the path
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]

        return max_flow

In [58]:
g = Graph(grid)

In [59]:
g.edmonds_karp(0, len(grid)-1)

Broken component size: 800 715


3

In [60]:
800*715

572000